# ImgLib2 

Example 07: Interpolation

## Introduction

Interpolation is a basic operation required in many image processing tasks. In the terminology of ImgLib2 it means to convert a [**`RandomAccessible`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html) into a [**`RealRandomAccessible`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccessible.html) which is able to create a [**`RealRandomAccess`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccess.html). It can be positioned at real coordinates instead of only integer coordinates and return a value for each real location. 

Currently, three interpolation schemes are available for ImgLib2:
  * Nearest neighbor interpolation (also available for any kind of data that can return a nearest neighbor like sparse datasets)
  * Linear interpolation
  * Lanczos interpolation

In the example, we magnify a given real interval in the [**`RealRandomAccessible`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccessible.html) which is based on the interpolation on an [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html) and compare the results of all three interpolation methods.

Below is the utility method for a given interval magnification.

In [1]:
import net.imglib2.type.Type
import net.imglib2.RealRandomAccessible
import net.imglib2.RealInterval
import net.imglib2.img.ImgFactory
import net.imglib2.img.Img

mag = new Object() {
  /**
   * Compute a magnified version of a given real interval
   *
   * @param <T> value type
   * @param source - the input data
   * @param interval - the real interval on the source that should be
   * magnified
   * @param factory - the image factory for the output image
   * @param magnification - the ratio of magnification
   * @return - an Img that contains the magnified image content
   */
  public <T extends Type<T>> Img<T> magnify(
    RealRandomAccessible<T> source,
    RealInterval interval,
    ImgFactory<T> factory,
    double magnification) {
    
    numDimensions = interval.numDimensions()
    
    // compute the number of pixels of the output and the size of the real interval
    pixelSize = new long[numDimensions]
    intervalSize = new double[numDimensions]
    
    for (int d = 0; d < numDimensions; ++d) {
      intervalSize[d] = interval.realMax(d) - interval.realMin(d)
      pixelSize[d] = Math.round(intervalSize[d] * magnification) + 1
    }
    
    // create the output image
    output = factory.create(pixelSize, source.realRandomAccess().get())
    
    // cursor to iterate over all pixels
    cursor = output.localizingCursor()
    
    // create a RealRandomAccess on the source (interpolator)
    realRandomAccess = source.realRandomAccess()
    
    // the temporary array to compute the position
    tmp = new double[numDimensions]
    
    // for all pixels of the output image
    while (cursor.hasNext()) {
      cursor.fwd()
      
      // compute the appropriate location of the interpolator
      for (int d = 0; d < numDimensions; ++d) {
        tmp[d] = cursor.getDoublePosition(d) / output.realMax(d) * intervalSize[d] + interval.realMin(d)
      }
      
      // set the position
      realRandomAccess.setPosition(tmp)
      
      // set the new value
      cursor.get().set(realRandomAccess.get())
    }
    
    return output
  }
}

Script1​$1​@79ecdf7f

The below code uses three different interpolators to 10x magnify a small area.

In [5]:
import io.scif.img.ImgOpener
import net.imglib2.FinalRealInterval
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.interpolation.randomaccess.LanczosInterpolatorFactory
import net.imglib2.interpolation.randomaccess.NLinearInterpolatorFactory
import net.imglib2.interpolation.randomaccess.NearestNeighborInterpolatorFactory
import net.imglib2.img.array.ArrayImgFactory
import net.imglib2.view.Views

enum InterpolationType {
  NEAREST_NEIGHBOR_INTERPOLATION,
  NLINEAR_INTERPOLATION,
  LANCZOS_INTERPOLATION
}

// open with ImgOpener as a FloatType
image = new ImgOpener().openImgs("http://samples.fiji.sc/tutorials/DrosophilaWing.tif", new FloatType()).get(0)

// define the area in the interpolated image
double[]min = [105.12, 40.43]
double[]max = [129.56, 74.933]

interval = new FinalRealInterval(min, max)

imgs = [:]
for (interpolant_type in
    [InterpolationType.NEAREST_NEIGHBOR_INTERPOLATION,
    InterpolationType.NLINEAR_INTERPOLATION,
     InterpolationType.LANCZOS_INTERPOLATION])
{
    // create a RandomAccessible using the factory and views method
    // it is important to extend the image first, the interpolation scheme might
    // grep pixels outside of the boundaries even when locations inside are queried
    // as they integrate pixel information in a local neighborhood - the size of
    // this neighborhood depends on which interpolator is used
    switch (interpolant_type) {
      case InterpolationType.NEAREST_NEIGHBOR_INTERPOLATION:
        // create an InterpolatorFactory RealRandomAccessible using nearst neighbor interpolation
        nn_inter_factory = new NearestNeighborInterpolatorFactory<FloatType>()
        nn_interpolant = Views.interpolate(Views.extendMirrorSingle(image), nn_inter_factory)
        img = mag.magnify(nn_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
        break
      case InterpolationType.NLINEAR_INTERPOLATION:
        // create an InterpolatorFactory RealRandomAccessible using linear interpolation
        linear_inter_factory = new NLinearInterpolatorFactory<FloatType>()
        nlinear_interpolant = Views.interpolate(Views.extendMirrorSingle(image), linear_inter_factory)
        img = mag.magnify(nlinear_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
        break
      case InterpolationType.LANCZOS_INTERPOLATION:
        // create an InterpolatorFactory RealRandomAccessible using lanczos interpolation
        lanczos_inter_factory = new LanczosInterpolatorFactory<FloatType>()
        lanczos_interpolant = Views.interpolate(Views.extendMirrorSingle(image), lanczos_inter_factory)
        img = mag.magnify(lanczos_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
        break
      default:
        nn_inter_factory = new NearestNeighborInterpolatorFactory<FloatType>()
        nn_interpolant = Views.interpolate(Views.extendMirrorSingle(image), nn_inter_factory)
        img = mag.magnify(nn_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
    }
    imgs.put(interpolant_type, img)
}
[imgs]

NEAREST​_NEIGHBOR​_INTERPOLATION NLINEAR​_INTERPOLATION LANCZOS​_INTERPOLATION <img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAPUAAAFaCAYAAADGjBvqAAAPp0lEQVR42u3dYWiV9RvHYYmIwqQoWGgkI0IIjSLCmGGkVCQsgorChlLSoLAwSJJNEBIVZcbExDAxluQwVCY1FiqGSoXiRjLFWEPZwslER7KNhUXtz3kng+Db9vzZsuuBz9tzds55rmdvbu7fpOH/8/XXX39Fjdfl7/t3fi+uv78m3agfLL0pi+7PP/8cl1wuqKF2QQ011C6ooYbaBTXUULughhpqF9RQu6CGGmoX1FBD7YIaaqhdUP/dVTSaP/74Y0JnrNMFNdQuqKGG2gU11FC7oIYaahfUUEPtghpqF9RQQ+2CGmqoXVBDDbXrv4c6vXlTDOOF//fff48ar9dzQQ011C6ooYbaBTXUULughhpqqKGGGmqooYbaBTXUULughhpqF9RQQw011BOtFOtvv/0Wlb5v0QsZJ/r9ciMdaQw11FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdQTF/V4HQFb9Hhl0V27di0q/RxFv56Fh//ehwnUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FD/+1GPF8Ki3zfFNV6ln6PocdIbBVfR30vRD9nSbwc11FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdQTF3V6Fb2I70YpXShYNOqJPiZa9PsWjbq/vz9qYGAg6tKlS1G9vb1QQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEN9A6BOr6IX8Y3X2GmKdWhoqNDS9y36KOCixyGLvg/S76Wvry+qpaUlavXq1VHV1dVRL774ItRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUP+HUE/0RYbpzVF0RR9lW/QYZvq+g4ODUT09PVFnzpyJam1tjTpx4kTUkSNHog4cOBDV3NwctXv3bqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooZ7AqCf6YrqixxxThEUfjVv0WGfRiyDTz9Hd3R21Z8+eqNra2qjXXnstat68eVGzZs2KKi8vj7rvvvuiysrKoIYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGG+gZAPdErGk3RiwyLHndNEaZHsaZHu6aL/bq6uqLOnj0b1d7eHnX8+PGo0rhmUn19fdT7778fVVlZCTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011DcA6vQqeswx/fsm+oLCFE1vb29UZ2dn1NGjR6MaGhqiampqohYtWhQ1f/78qIqKiqgHH3wwasaMGVHTp0+PmjlzZtSTTz4JNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUExh10VjTq+hFgenY5NDQUFT6einC/fv3R6XjhgsXLox67rnnolI0Dz30UFSKYdq0aVF333131OTJk6OmTJkSlS4oXLp0adSyZcughhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYZ6AqMueky06EV8ly9fjkrHJk+dOhV1+PDhqPTI1s2bN0etXLkyKh3XTFu/fn3UqlWrol599dWoZ599Nurll1+OSl8vraqqKmrNmjVRdXV1UEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDfQOgTkvHKy9evBi1Y8eOqBUrVkQtXrw46vnnn4+aM2dOVDpe+cADD0Sl45XpOOTNN99caGVlZVEp/tmzZ0el7/vMM89EpQ+7pqamOKihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooS4OddEIiz4qNkU9ODgYdfXq1ah0AWD6MOnq6orq6OiISsdTjxw5EtXY2BiVjp0uWLAg6p577omaOnVq1Ny5c6Oqq6ujli9fHpW+Xm1tbdS2bduiPv74Y6ihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooS4QddFHz6ZHxaZHwPb390f19PREpQsF29raovbu3Ru1bt26qCVLlkSl46T33ntvVHl5eVR65G1lZWXUK6+8EvXOO+9EpUf3pg+n7du3R23ZsiUqHRMtYU365JNPoIYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGegyo06NhR5aOYaZHxaZo0i8rXQD46KOPRqU3ebqYbvLkyVG33XZbVLqw76677opKxzUffvjhqBkzZkQ99dRTUe+9917Uxo0bozZs2BCVHim7adOmqPQhkbZ161aooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKEeA+qRi/3Ssc6+vr6ogwcPRqV/9M6dOwstfd9ly5ZFpYvu0qNO0/dNb/K333476sMPP4xKMaxduzYq/byvv/56VE1NTVTRY51p9fX1UeniwdWrV0MNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDfUYUF+4cGH4+n7++eeo1tbWqKampqgPPvggat68eVHpmOOtt94aNWnSpKh0XDMdm3z66aejHnnkkaiqqqqodBFfiiZ9vfQ+eOutt6IqKiqi0odi+hBLx0TTxYjpw7j03UANNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDfXoUY8cg3vhhRei7r///qipU6dGpQv20sV5jz32WNSsWbOi0kWBKf4pU6ZE3XHHHVFPPPFEVHV1dVR6E73xxhtR6cMk/d3Sh2J6NG46JpoeKZuOiaYPicbGxqjSEkWooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKEePepp06YNX196s91yyy1RKdabbropKh3/nD17dlQ6hrl06dKodHHe8uXLo9KH7LvvvhtVV1cXtWrVqqhFixZFpWOiH330UVTpyNakdevWRaXjzOkizX379kUdOHAg6tChQ1Gl14QaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGurRox451pmimTlzZlRZWVnUnXfeGTV//vyodKFbunCudERo0ksvvRSVLlBMF+ylC/vWrFkTlY5Nfvnll1G7d+8utJMnT0adPn06qqOjI+rrr7+OOnPmTNRPP/0U1dLSEtXc3Aw11FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRjQH377bcPX1+6iC9d/DZ37tyodNFdfX19VDrmWFlZGVVeXh71+OOPR6ULANPPmyJMSxfdffbZZ1Hbtm2L2rVrV9QPP/wQlY5hpu9bQpOUHgnd2dkZ1dvbG9XT0wM11FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRQQw011FBDDTXUUEMNNdRjQD19+vTh65szZ05UumCvtrY2qqamJirFnx7tWlFREfXmm29GpZ83XQCYlmJtbW2NStF89dVXUenYZHt7e1Q6rnns2LGo8+fPR506dSoq/f6OHz9eaIcPH4YaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGmqooYYaaqihhhpqqKGGGuoxoF6wYMHw9aUL+9IFdulRoulRrAsXLoxKj1hNFw82NDREpV9+W1tbVLo4L73Z0kV3586di0oxdHV1FdqlS5eiLl++HNXd3R2Vfs/fffdd1J49

The nearest neighbor interpolation is the fastest to compute and is the most versatile as it requires no computation, but just a lookout. The result is, however, very pixelated. The linear interpolation produces reasonable results and computes quite fast. The Lanczos interpolation shows the most visually pleasing results, but also introduces slight artifacts in the background.